<H3> Expedia API - rates retrieval </h3>


First we need to generate an auth header for the API. 
This function can be checked against https://developers.expediagroup.com/rapid/tools/signature-generator


In [409]:
# generate authorisation header - this is verified against expedia's EPS signature generator at https://developers.expediagroup.com/rapid/resources/tools/signature-generator

import time, hashlib

def auth_header(API_KEY, SHARED_SECRET):
    ts = str(int(time.time()))
    sig = hashlib.sha512((API_KEY + SHARED_SECRET + ts).encode("utf-8")).hexdigest()
    return f"EAN APIKey={API_KEY},Signature={sig},timestamp={ts}"


In [410]:
# check API auth header 

API_KEY = "14v1pb3juv5gr8i0hd34opr4p7"
SHARED_SECRET = "3lqqfdrkctce9"
auth_header(API_KEY, SHARED_SECRET)

'EAN APIKey=14v1pb3juv5gr8i0hd34opr4p7,Signature=ba5122b11238987cf4961a044c34218d662daec35c447255747e519fa467bcf8d7a65d744c0fac9eae790928c46b419e2b4045a98b3b5c06e4e58bad0b6c3082,timestamp=1759257225'

Next, a function to retrieve pricing for a given hotel

In [411]:
'''function to retrieve pricing for a given hotel'''
import os
import time
import hashlib
import json
import requests

def price_check(API_BASE, Property_ID, checkin, checkout, occupancy, rate_plan_count, rate):
    '''rate options: priv_pkg, pub_prepay, pub_hotelpay, mkt_prepay, mkt_hotelpay
    
    API_BASE: Url for API, usually "https://test.ean.com/"
    Property_ID: Expedia database ID for the property
    checkin: start date
    checkout: end date
    occupancy: how many people to sleep
    rate_plan_count: How many rates (ie room types) to show
    '''
    # --- BUILD AUTH HEADER ---
    auth = auth_header(API_KEY, SHARED_SECRET)

    headers = {
        "Authorization": auth,
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "User-Agent": "RapidTest/0.1",
        "Customer-Ip": "8.8.8.8"  # any valid public IP
    }

    rates = {
        'pub_prepay': {'payment_terms': None, 'partner_point_of_sale': "SA_MOD"},
        'pub_hotelpay': {'payment_terms': None, 'partner_point_of_sale': "HC_MOD"},
        'mkt_prepay': {'payment_terms': "EPSMOR_MARKET", 'partner_point_of_sale': "EPSMOR_SA_MARKET"},
        'mkt_hotelpay': {'payment_terms': "CCC_MARKET", 'partner_point_of_sale': "CCC_SA_HC_MARKET"},
        'priv_pkg': {'payment_terms': "PKG_EPSMOR", "partner_point_of_sale": "EPSMOR_PKG_EXPPKG_MOD"}
        }

    params = {
        "checkin": checkin,
        "checkout": checkout,
        "currency": "USD",
        "country_code": "US",
        "language": "en-US",
        "occupancy": occupancy,
        "property_id": Property_ID,
        #   The ID of the property you want to search for. You can provide 1 to 250 property_id parameters.
        "rate_plan_count": rate_plan_count,
        "sales_channel": "website",
        "sales_environment": "hotel_package",
        # '''You must provide the sales environment in which rates will be sold. EPS dynamically provides the best content for optimal conversion. If you have a sales environment that is not currently supported in this list, please contact our support team.<br>
        #   * `hotel_package` - Use when selling the hotel with a transport product, e.g. flight & hotel.
        #   * `hotel_only` - Use when selling the hotel as an individual product.
        #   * `loyalty` - Use when you are selling the hotel as part of a loyalty program and the price is converted to points.'''
        "payment_terms": rates[rate]["payment_terms"],
        "partner_point_of_sale": rates[rate]["partner_point_of_sale"]
        }

    resp = requests.get(f"{API_BASE}v3/properties/availability",
                        headers=headers, params=params, timeout=30)

    # DEBUG: Print to terminal pricing
    '''
    print("Status:", resp.status_code)
    if resp.ok:
        data = resp.json()
        # Print a simple summary
        for prop in data:
            print("Property:", prop.get("property_id"), "status:", prop.get("status"))
            
            # organise rooms alphabetically
            rooms = sorted(prop.get("rooms", []), key=lambda r: (r.get("room_name") or "").lower())
            
            for room in rooms:
                print("  Room:", room.get("room_name"))
                for rate in room.get("rates", []):
                    occ = rate.get("occupancy_pricing", {}).get(occupancy, {})
                    total = (occ.get("totals") or {}).get("inclusive", {})
                    price = (total.get("request_currency") or {}).get("value")
                    currency = (total.get("request_currency") or {}).get("currency")
                    print("    Rate:", rate.get("id"), "| Price:", price, currency,
                        "| Refundable:", rate.get("refundable"))
    else:
        print("Error:", resp.text)
    '''


    if resp.ok:
        return resp
    
    else: 
        print("Error:", resp.text)
        return 0
    

filename = "data/rate_info.json"
def save_result(response, filename):
    with open(filename, "w") as f:
        json.dump(response.json(), f, indent=4)  # Pretty-print with indentation

In [412]:
'''Function to organise rates into a dataframe for easy viewing'''
import pandas as pd

def build_rates_dataframe(jsons, labels):
    """
    Parameters
    ----------
    jsons : list of dict
        Each dict should have the same structure you posted (contains rooms with 'room_name' and 'rates').
    labels : list of str
        Names to assign to each JSON's prices column. Must match length of jsons.
        
    Returns
    -------
    pd.DataFrame
        DataFrame with 'room_name' from the first JSON and one column per label
        showing the cheapest 'inclusive' total price (request_currency.value) for that room.
    """
    if len(jsons) != len(labels):
        raise ValueError("`jsons` and `labels` must have the same length.")
    
    def _extract_price_from_rate(rate):
        """Safely pull the inclusive request-currency value (as float) for occupancy '2' from a rate."""
        try:
            return float(
                rate["occupancy_pricing"]["2"]["totals"]["inclusive"]["request_currency"]["value"]
            )
        except (KeyError, TypeError, ValueError):
            return None

    # --- Helper to extract {room_name: cheapest_price} mapping ---
    def extract_room_prices(data):
        mapping = {}
        for room in data.get("rooms", []):
            room_name = room.get("room_name")
            if not room_name:
                continue

            prices = []
            for rate in room.get("rates", []) or []:
                price = _extract_price_from_rate(rate)
                if price is not None:
                    prices.append(price)

            mapping[room_name] = min(prices) if prices else None
        return mapping
    
    # --- Build initial DataFrame from first JSON ---
    first_json = jsons[0]
    rooms = sorted(first_json.get("rooms", []), key=lambda r: (r.get("room_name") or "").lower())
    df = pd.DataFrame([{"room_name": r.get("room_name")} for r in rooms])

    # Add columns for each JSON/label
    for data, label in zip(jsons, labels):
        price_map = extract_room_prices(data)
        df[label] = df["room_name"].map(price_map)
    
    return df

In [430]:
API_BASE = "https://test.ean.com/"

PROPERTY_ID = ["114116366"]    # Sentral Gaslamp Quarter San Diego
CHECKIN = "2025-11-04"
CHECKOUT = "2025-11-08"
OCCUPANTS = "2"
RATES = 3

prv_rate = price_check(API_BASE, PROPERTY_ID, CHECKIN, CHECKOUT, OCCUPANTS, RATES, "priv_pkg")
pub_rate = price_check(API_BASE, PROPERTY_ID, CHECKIN, CHECKOUT, OCCUPANTS, RATES, "pub_prepay")
mkt_rate = price_check(API_BASE, PROPERTY_ID, CHECKIN, CHECKOUT, OCCUPANTS, RATES, "mkt_prepay")

print(mkt_rate.json())

filename = "data/rate_info.json"
save_result(prv_rate, filename)

[{'property_id': '114116366', 'status': 'available', 'rooms': [{'id': '326351700', 'room_name': 'Studio Accessible (ADA)', 'rates': [{'id': '400884669', 'status': 'available', 'available_rooms': 4, 'refundable': True, 'member_deal_available': True, 'sale_scenario': {'package': True, 'member': False, 'corporate': False, 'distribution': False}, 'merchant_of_record': 'expedia', 'amenities': {'2192': {'id': '2192', 'name': 'Free WiFi'}}, 'links': {'payment_options': {'method': 'GET', 'href': '/v3/properties/114116366/payment-options?token=REhZAQsABAoHQhMHUlNCF1ELUj1DVhdLBQMES1REClxLQg5cRR8WBlIBSxZdCFRpTQxRWVUAU1BSWFQDH1YDBVEbAwsECh0MDgZVSFAGUQYMCQQDDVZXBB8IVURWUAdWRDtZVWgTVgBXRgBccWBiJH0vdENSQkoRXA5WRGsVVhdKX19db1xXWVNcAARXVgYLFFANU1EVVQxTDEwKUVZRH1QBAQYFVgMNBFMAUBJQQUEVX1wHEGkLEwwMTF0fDR0BFUQHRBdfVlY-XV0OB1cECAoGVFMVUFFQABkEUAEHSAoEUwBJUwFSWVsMAgVWU1MHHkdXEUVQUV5GB0UMAhoVUAoUEkcWQkUPXgNDOVtJAEcARQhWAmhbV1NXCV4vcmoneCR5NxJVVQxQE1lNUQFsRAhcVgpWVQENSwZfFAIJMQUOQQt1UVUVAyIEUhYDVwZcUgBQV1RpFFB

In [431]:
a = 0
build_rates_dataframe(
    (
        prv_rate.json()[a],
        pub_rate.json()[a],  
        mkt_rate.json()[a]), 
    
    (
        "priv_rate", 
        "pub_rate", 
        "mkt_rate")
    )

,room_name,priv_rate,pub_rate,mkt_rate
0,Studio Accessible (ADA),1037.16,1343.93,1343.93
1,"Studio, 1 King Bed",1037.16,1343.93,1343.93


Search for properties around a given location. Given Lat, Lon and a search radius

In [415]:
import requests, time, hmac, hashlib

def region_properties_near(lat, lon, radius_km=1, limit=100):
    url = f"{API_BASE}/v3/regions"
    headers = {
        "Authorization": auth_header(API_KEY, SHARED_SECRET),
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "User-Agent": "RapidTest/0.1",
        "Customer-Ip": "203.0.113.9",
    }
    params = {
        "area": f"{int(radius_km)},{lat},{lon}",  # radius must be whole km
        "include": "property_ids", # add "property_ids_expanded" for more properties, up to 30km out
        "limit": limit,
        "language": "en-US",
        "supply_source": "expedia",
    }
    r = requests.get(url, headers=headers, params=params, timeout=30)
    r.raise_for_status()
    # flatten out so we just have a list of properties:
    property_ids = []
    for region in r.json():
        for p_id in region.get('property_ids', {}):
            property_ids.append(p_id)
    return property_ids

In [416]:
# search for properties around RAI Amsterdam
Lat = 52.34126923037023 
Long = 4.888663878076148
radius_km = 10

response = region_properties_near(Lat, Long, radius_km)
with open("data/property_search.json", "w") as f:
        json.dump(response, f, indent=4)

response

['6269644',
 '912532',
 '29305305',
 '26549242',
 '16042031',
 '1364271',
 '44992736',
 '23051797',
 '7439344',
 '24810',
 '19475818',
 '8096',
 '24092999',
 '570360',
 '7420731',
 '790890',
 '15546981',
 '3575922',
 '3525665',
 '44770485',
 '20129749',
 '11091426',
 '27392764',
 '23568977',
 '5030792',
 '46355261',
 '21854580',
 '5768911',
 '20416728',
 '46345458',
 '43466274',
 '46345462',
 '866153',
 '54245',
 '866154',
 '2256225',
 '3445436',
 '33300767',
 '20517439',
 '22227028',
 '43997797',
 '44993600',
 '182657',
 '2599265',
 '22992095',
 '33748407',
 '54250',
 '1444010',
 '9663198',
 '21956589',
 '6048718',
 '5282002',
 '4736208',
 '4344128',
 '22446245',
 '1042164',
 '1042165',
 '3832233',
 '1042163',
 '5026288',
 '66248',
 '1364283',
 '1364282',
 '1364284',
 '1445350',
 '3190188',
 '46345434',
 '3605893',
 '1572366',
 '21645182',
 '1584349',
 '1400487',
 '1364288',
 '9027317',
 '8264502',
 '46208697',
 '3681',
 '8280198',
 '11756669',
 '18033666',
 '27049',
 '4459442',
 '110

In [417]:
len(response)

7728

In [418]:
'''Get details for given properties - up to 250 at a time'''

def fetch_property_summaries(property_ids, includes=("name","address","location", "statistics")):
    if not property_ids:
        return {}
    url = f"{API_BASE}/v3/properties/content"
    headers = {
        "Authorization": auth_header(API_KEY, SHARED_SECRET),
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "User-Agent": "RapidTest/0.1",
        "Customer-Ip": "203.0.113.9",
    }
    params = [("language","en-US"), ("supply_source","expedia")]
    for inc in includes:
        params.append(("include", inc))
    for pid in property_ids:
        params.append(("property_id", pid))
    r = requests.get(url, headers=headers, params=params, timeout=30)
    
    r.raise_for_status()
    return r.json()  # keyed by property_id

# fetch_property_summaries(response[0]['property_ids'])
# fetch_property_summaries(response[:10])
summary = fetch_property_summaries(["7439344"])
summary

{'7439344': {'property_id': '7439344',
  'name': 'Hotel Asselt',
  'address': {'line_1': 'Oude Baan 6',
   'line_2': 'Swalmen',
   'city': 'Swalmen',
   'postal_code': '6071 NP',
   'country_code': 'NL',
   'obfuscation_required': False,
   'localized': {'links': {'en-US': {'method': 'GET',
      'href': 'https://api.ean.com/v3/properties/content?language=en-US&property_id=7439344&include=address&supply_source=expedia'}}}},
  'location': {'coordinates': {'latitude': 51.225261, 'longitude': 6.026503},
   'obfuscation_required': False},
  'statistics': {'2515': {'id': '2515',
    'name': 'Year Built - 1849',
    'value': '1849'},
   '52': {'id': '52', 'name': 'Total number of rooms - 29', 'value': '29'},
   '54': {'id': '54', 'name': 'Number of floors - 2', 'value': '2'}}}}

In [419]:
# find out info about a given region
region_id = 11396
headers = {
    "Authorization": auth_header(API_KEY, SHARED_SECRET),
    "Accept": "application/json",
    "Accept-Encoding": "gzip",
    "User-Agent": "RapidTest/0.1",
    "Customer-Ip": "203.0.113.9",
}
# for properties
params = {
    "include": "property_ids",      # or "details" or "property_ids_expanded"
    "language": "en-GB",            # pick from the allowed list
    "supply_source": "expedia"
}

# or for details:
params = {"include": "details", "language": "en-GB", "supply_source": "expedia"}


resp = requests.get(f"{API_BASE}/v3/regions/{region_id}",
                    headers=headers, params=params, timeout=30)
resp.raise_for_status()
data = resp.json()
print(data)

{'id': '11396', 'type': 'city', 'name': 'Aalsmeer', 'name_full': 'Aalsmeer, North Holland, Netherlands', 'country_code': 'NL', 'country_subdivision_code': 'NL-NH', 'coordinates': {'center_longitude': 4.761922, 'center_latitude': 52.262165, 'bounding_polygon': {'type': 'Polygon', 'coordinates': [[[4.670293, 52.230795], [4.671588, 52.231466], [4.672893, 52.232142], [4.673407, 52.232409], [4.673647, 52.232533], [4.67397, 52.232701], [4.674137, 52.232788], [4.675096, 52.233283], [4.675792, 52.233649], [4.676331, 52.233925], [4.676389, 52.233955], [4.676853, 52.234242], [4.677003, 52.234338], [4.677074, 52.234387], [4.677327, 52.234562], [4.677769, 52.234931], [4.677899, 52.235053], [4.677996, 52.235151], [4.678025, 52.23518], [4.678174, 52.23534], [4.678279, 52.235452], [4.678488, 52.235689], [4.678733, 52.236026], [4.679067, 52.236502], [4.67942, 52.237011], [4.679449, 52.237053], [4.679632, 52.23732], [4.679687, 52.237398], [4.67972, 52.237446], [4.67985, 52.237634], [4.680019, 52.237876

In [420]:
'''find properties within a given polygon from the API'''
def polygon_property_ids(
    polygon_geojson: Dict[str, Any],
    *,
    include: str = "property_ids",           # per docs
    supply_source: Optional[str] = "expedia",# "expedia" or "vrbo"
    customer_session_id: Optional[str] = None,
    endpoint_path: str = "v3/properties/geography",
    timeout: int = 30
) -> List[str]:
    """
    Calls the polygon API with the given GeoJSON polygon and returns a flat list of property IDs.
    Only uses the query params and headers specified in the provided docs.
    """
    url = f"{API_BASE}{endpoint_path}"

    headers = {
        "Authorization": auth_header(API_KEY, SHARED_SECRET),
        "Accept": "application/json",
        "Accept-Encoding": "gzip",
        "User-Agent": "TravelNow/3.30.112",  # example format per docs
    }
    if customer_session_id:
        headers["Customer-Session-Id"] = customer_session_id

    # Query-string parameters per docs from expedia
    params = {"include": include}
    if supply_source:
        params["supply_source"] = supply_source
    # (billing_terms / partner_point_of_sale / payment_terms / platform_name can be added if needed - i don't think they are very important

    resp = requests.post(url, headers=headers, params=params, json=polygon_geojson, timeout=timeout)
    resp.raise_for_status()
    data = resp.json()

    # Response example shows an object keyed by property_id: {"9777": {"property_id":"9777"}, ...}
    # this might break here if the response is different to the example
    # Extract the keys; fall back to nested "property_id" if needed.
    prop_ids = set()
    if isinstance(data, dict):
        for k, v in data.items():
            if isinstance(k, str):
                prop_ids.add(k)
            if isinstance(v, dict) and "property_id" in v:
                prop_ids.add(str(v["property_id"]))
    return sorted(prop_ids)


In [421]:
'''generate a circle of points on the earth surface of radius 
   this can be viewed by pasting the results of to_geojson_string into geojson.io'''
import math
from typing import Dict, Any, List, Tuple

EARTH_RADIUS_M = 6371008.8  # mean Earth radius (meters)

def destination_point(lat_deg: float, lon_deg: float, bearing_deg: float, distance_m: float) -> Tuple[float, float]:
    """
    Spherical Earth 'destination point' calculation.
    Returns (lat_deg, lon_deg).
    """
    φ1 = math.radians(lat_deg)
    λ1 = math.radians(lon_deg)
    θ = math.radians(bearing_deg)
    δ = distance_m / EARTH_RADIUS_M  # angular distance

    sinφ1, cosφ1 = math.sin(φ1), math.cos(φ1)
    sinδ, cosδ = math.sin(δ), math.cos(δ)
    sinθ, cosθ = math.sin(θ), math.cos(θ)

    sinφ2 = sinφ1 * cosδ + cosφ1 * sinδ * cosθ
    φ2 = math.asin(max(-1.0, min(1.0, sinφ2)))  # clamp

    y = sinθ * sinδ * cosφ1
    x = cosδ - sinφ1 * sinφ2
    λ2 = λ1 + math.atan2(y, x)

    # Normalize lon to [-180, 180)
    lon2 = (math.degrees(λ2) + 540) % 360 - 180
    lat2 = math.degrees(φ2)
    return lat2, lon2

def circle_polygon_geojson(
    center_lat: float,
    center_lon: float,
    radius_m: float,
    n_points: int = 64
) -> Dict[str, Any]:
    """
    Generate a GeoJSON Polygon

    Args:
        center_lat: Latitude in degrees.
        center_lon: Longitude in degrees.
        radius_m: Radius in meters.
        n_points: Number of vertices around the circle (3..200). The polygon
                  will be closed by repeating the first coordinate at the end.

    Returns:
        A dict GeoJSON Polygon with coordinates as [ [ [lon, lat], ... ] ].
    """
    if radius_m <= 0:
        raise ValueError("radius_m must be > 0")
    n = max(3, min(200, int(n_points)))

    coords: List[List[float]] = []
    # Sample bearings 0..360 (exclusive), then close ring by repeating first point
    for i in range(n):
        bearing = (360.0 * i) / n
        lat, lon = destination_point(center_lat, center_lon, bearing, radius_m)
        coords.append([lon, lat])  # GeoJSON is [lon, lat]

    # Close the polygon
    coords.append(coords[0])

    return {
        "type": "Polygon",
        "coordinates": [coords],
    }


import json
from typing import Any, Dict

def to_geojson_string(geo: Dict[str, Any], pretty: bool = True) -> str:
    """
    Convert a Python GeoJSON-like dict into a JSON string ready to paste
    into geojson.io for debugging etc
    """
    if pretty:
        return json.dumps(geo, indent=2)
    return json.dumps(geo, separators=(",", ":"))

# Example usage:
# polygon = circle_polygon_geojson(37.208957, -93.292000, radius_m=5000, n_points=120)
# ids = polygon_property_ids(polygon, include="property_ids", supply_source="expedia")
# print(ids)

In [436]:
# RAI Amsterdam
lat = 52.34687283454716
lng = 4.88998731115722

# San Diego
lat = 32.7057654197668
lng = -117.16013425869112

polygon = circle_polygon_geojson(lat, lng, radius_m=5000, n_points=120)
ids = polygon_property_ids(polygon, include="property_ids", supply_source="expedia")
print(ids)



['10001437', '10003291', '100374314', '100924317', '101084828', '10108642', '102182794', '102234538', '10237269', '10351228', '103987320', '104016817', '104885966', '105459259', '105473178', '105481776', '106165718', '106252293', '1064443', '1064570', '1065506', '10655687', '1074535', '107550957', '108197757', '108641880', '108733619', '108881', '1090002', '109276282', '109530737', '1095679', '109643192', '110299112', '110341567', '111526584', '111827598', '1121636', '11229106', '112541650', '11293', '112966921', '11301676', '1130523', '1130576', '1142030', '1142033', '1142035', '1142038', '1142040', '1142041', '114422137', '114566502', '11500383', '11513005', '11587613', '116216082', '116595330', '116678961', '1169547', '117308844', '1175835', '1177', '118036278', '119440871', '1205337', '120582455', '12063575', '120696750', '12071757', '1211201', '12306987', '12360551', '12375980', '12449223', '12579342', '12591593', '12593271', '12709785', '1276295', '12942259', '13047', '13063045',

In [438]:
properties = {}
priv_prices = {}
mkt_prices = {}
count = len(ids)
step = 250

for start in range(0, count, step):
    end = min(count, start + step)
    print(start, end - 1)
    properties.update(fetch_property_summaries(ids[start:end]))

0 249
250 495


In [439]:
summary_flat = []
for i, property in enumerate(properties.values()):
    summary_flat.append(property)

df = pd.DataFrame(summary_flat)
room_numbers = []
floor_counts = []
founding_dates = []
for property in summary_flat:
    room_numbers.append(property.get("statistics", {}).get("52", {}).get("value", ""))
    floor_counts.append(property.get("statistics", {}).get("54", {}).get("value", ""))
    founding_dates.append(property.get("statistics", {}).get("2515", {}).get("value", ""))

df["rooms"] = room_numbers
# df["floors"] = floor_counts
# df["founding_date"] = founding_dates


df


,property_id,name,address,location,statistics,rooms
0,22998,Sofitel Legend The Grand Amsterdam,"{'line_1': 'Oudezijds Voorburgwal 197', 'city'...","{'coordinates': {'latitude': 52.371015, 'longi...","{'2515': {'id': '2515', 'name': 'Year Built - ...",178
1,10237269,ClinkNOORD - Hostel,"{'line_1': 'Badhuiskade 3', 'city': 'Amsterdam...","{'coordinates': {'latitude': 52.384208, 'longi...","{'2515': {'id': '2515', 'name': 'Year Built - ...",213
2,21438064,YAYS Amsterdam East by Numa,"{'line_1': 'Zeeburgerpad 28-29', 'city': 'Amst...","{'coordinates': {'latitude': 52.36696, 'longit...","{'2515': {'id': '2515', 'name': 'Year Built - ...",15
3,1772121,Sphinx Hotel,"{'line_1': 'Weteringschans 82', 'city': 'Amste...","{'coordinates': {'latitude': 52.359328, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",22
4,1524945,"Max Brown Hotel Museum Square, part of Sircle ...","{'line_1': 'Jan Luykenstraat, 44', 'city': 'Am...","{'coordinates': {'latitude': 52.36, 'longitude...","{'52': {'id': '52', 'name': 'Total number of r...",64
...,...,...,...,...,...,...
491,32099928,SUPPER Hotel,"{'line_1': '462 Singel', 'city': 'Amsterdam', ...","{'coordinates': {'latitude': 52.367646, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",27
492,6041160,Hotel Titus,"{'line_1': 'Leidsekade 74', 'city': 'Amsterdam...","{'coordinates': {'latitude': 52.365055, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",19
493,8439319,Owl Hotel,"{'line_1': 'Roemer Visscherstraat 1', 'city': ...","{'coordinates': {'latitude': 52.361778, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",34
494,9458607,Hotel Café Restaurant Abina,"{'line_1': 'Amsterdamseweg 193', 'city': 'Amst...","{'coordinates': {'latitude': 52.311317, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",23


In [440]:
private_rates = []
hotel_private_prices = []

market_rates = []
hotel_market_prices = []

checkin = "2025-11-04"
checkout = "2025-11-08"
occupancy_key = '2'

for start in range(0, count, step):
    end = min(count, start + step)

    private_rate_query = price_check(API_BASE, ids[start:end], checkin, checkout, 2, 1, "priv_pkg").json()
    private_rates.extend(private_rate_query)

    market_rate_query = price_check(API_BASE, ids[start:end], checkin, checkout, 2, 1, "mkt_prepay").json()
    market_rates.extend(market_rate_query)


for rate in private_rates:
    best_private_rate = rate['rooms'][0]['rates'][0]['occupancy_pricing'][occupancy_key]["totals"]["inclusive"]["request_currency"]["value"]
    property_id = rate.get("property_id", "")

    hotel_private_prices.append({"property_id": property_id, "rate": best_private_rate})


for rate in market_rates:
    best_market_rate = rate['rooms'][0]['rates'][0]['occupancy_pricing'][occupancy_key]["totals"]["inclusive"]["request_currency"]["value"]
    property_id = rate.get("property_id", "")

    hotel_market_prices.append({"property_id": property_id, "rate": best_market_rate})


In [443]:
rate_map = {d['property_id']: float(d['rate']) for d in hotel_private_prices}
df["priv_rate"] = df["property_id"].map(rate_map)

rate_map = {d['property_id']: float(d['rate']) for d in hotel_market_prices}
df["mkt_rate"] = df["property_id"].map(rate_map)

df["Delta"] = df["priv_rate"] - df["mkt_rate"]

In [444]:
df.to_excel("properties.xlsx")
df

,property_id,name,address,location,statistics,rooms,priv_rate,mkt_rate,Delta
0,22998,Sofitel Legend The Grand Amsterdam,"{'line_1': 'Oudezijds Voorburgwal 197', 'city'...","{'coordinates': {'latitude': 52.371015, 'longi...","{'2515': {'id': '2515', 'name': 'Year Built - ...",178,2693.95,3388.06,-694.11
1,10237269,ClinkNOORD - Hostel,"{'line_1': 'Badhuiskade 3', 'city': 'Amsterdam...","{'coordinates': {'latitude': 52.384208, 'longi...","{'2515': {'id': '2515', 'name': 'Year Built - ...",213,428.39,491.83,-63.44
2,21438064,YAYS Amsterdam East by Numa,"{'line_1': 'Zeeburgerpad 28-29', 'city': 'Amst...","{'coordinates': {'latitude': 52.36696, 'longit...","{'2515': {'id': '2515', 'name': 'Year Built - ...",15,1207.57,1445.29,-237.72
3,1772121,Sphinx Hotel,"{'line_1': 'Weteringschans 82', 'city': 'Amste...","{'coordinates': {'latitude': 52.359328, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",22,NaN,NaN,NaN
4,1524945,"Max Brown Hotel Museum Square, part of Sircle ...","{'line_1': 'Jan Luykenstraat, 44', 'city': 'Am...","{'coordinates': {'latitude': 52.36, 'longitude...","{'52': {'id': '52', 'name': 'Total number of r...",64,518.44,594.92,-76.48
...,...,...,...,...,...,...,...,...,...
491,32099928,SUPPER Hotel,"{'line_1': '462 Singel', 'city': 'Amsterdam', ...","{'coordinates': {'latitude': 52.367646, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",27,664.51,717.18,-52.67
492,6041160,Hotel Titus,"{'line_1': 'Leidsekade 74', 'city': 'Amsterdam...","{'coordinates': {'latitude': 52.365055, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",19,435.36,501.34,-65.98
493,8439319,Owl Hotel,"{'line_1': 'Roemer Visscherstraat 1', 'city': ...","{'coordinates': {'latitude': 52.361778, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",34,NaN,NaN,NaN
494,9458607,Hotel Café Restaurant Abina,"{'line_1': 'Amsterdamseweg 193', 'city': 'Amst...","{'coordinates': {'latitude': 52.311317, 'longi...","{'52': {'id': '52', 'name': 'Total number of r...",23,NaN,NaN,NaN
